# Retour sur l&#8217;exemple

`scipy.integrate` fournit les outils pour intégrer des fonctions numériquement.
En particulier les fonction `trapz` et `simps` permettent d&#8217;intégrer des fonctions en utilisant respectivement la méthode des trapèzes et la méthode de Simpson.
*Exemple*\
Considérons l&#8217;exemple [[ex:hauteur]](#ex:hauteur) et supposons de vouloir calculer le nombre d&#8217;individus dont la hauteur soit comprise entre 1.8 et 1.9 mètres
On peut utiliser par exemple la formule de Simpson ou celle des trapèzes composite avec 100 sous-intervalles grâce à [`scipy.integrate`](https://docs.scipy.org/doc/scipy/tutorial/integrate.html):


In [0]:
import numpy as np
import math
from scipy.integrate import simps, trapz
from numpy.polynomial.legendre import leggauss
from tabulate import tabulate

# Définir la fonction

def N(h, M, hbar, sigma):
    return M / (sigma * np.sqrt(2. * np.pi)) * np.exp(-(h - hbar)**2 / (2 * sigma**2))


# Valeurs données
M = 100
hbar = 1.7
sigma = 0.1
N(hbar, M, hbar, sigma)
# Définir la plage pour l'intégration et le nombre de points
a = 1.8
b = 1.9

# Préparer la liste pour stocker les résultats
results = []
Ps=dict()
# Calculs avec la méthode des rectangles, des trapèzes et de Simpson
for num_intervalles in [1, 2, 4, 8, 16, 32, 64, 128]:
    num_points = num_intervalles+1
    h_values = np.linspace(a, b, num_points)
    function_values = N(h_values, M, hbar, sigma)

    integral_simpson = simps(function_values, h_values) # <1>
    integral_trapezoidal = trapz(function_values, h_values) # <2>
    midpoints = (h_values[:-1] + h_values[1:]) / 2
    integral_rectangle = np.sum(
        N(midpoints, M, hbar, sigma) * (h_values[1:] - h_values[:-1])) # <3>

    # Dictionnaire pour stocker les résultats
    integral_gauss = dict()
    for q in range(1, 6, 2): # <4>
        Ps[q] = math.ceil((q-1)/2) + 1 # <5>
        xi, w = leggauss(Ps[q]) # <6>

        # Concaténer les nouveaux points dans le tableau x_all
        integral = 0
        for k in range(len(h_values) - 1):
            x = (h_values[k+1] - h_values[k]) / 2 * xi + (h_values[k+1] + h_values[k]) / 2 # <7>
            # Utilisez le nouveau tableau x_new pour le calcul de l'intégrale
            J = (h_values[k+1] - h_values[k]) / 2 # <8>
            integral += np.sum(w * N(x, M, hbar, sigma) )*J # <9>

        integral_gauss[f'gauss_{q}'] = integral
    results.append([num_intervalles] +[v for k,v in integral_gauss.items()]+[         integral_rectangle, integral_trapezoidal, integral_simpson])

# Afficher les résultats avec tabulate
headers = ["Num intervalles", f"Gauss 1 ({Ps[1]})", f"Gauss 3 ({Ps[3]})", f"Gauss 5 ({Ps[5]})", "Rectangle (1)", "Trapezoidal (2)", "Simpson (3)"]
print(tabulate(results, headers=headers, tablefmt="grid", floatfmt=".9f")) # <8>



1. calcul de l'intégrale avec la méthode de Simpson
2. calcul de l'intégrale avec la méthode des trapèzes
3. calcul de l'intégrale avec la méthode des rectangles
4. boucle sur les degrés des polynomes que nous voulons intégrer exactement
5. calcul du nombre de points pour la quadrature de Gauss pour intégrer exactement un polynome de degré stem:[q]
6. calcul des points et des poids de la quadrature de Gauss sur stem:[[-1,1\]]
7. calcul des points de la quadrature de Gauss sur les intervalles stem:[[x_k,x_{k+1}\]]
8. calcul du jacobien stem:[J] pour le changement de variable de stem:[[-1,1\]] à stem:[[x_k,x_{k+1}\]]
9. accumulation des sommes pour le calcul de l'intégrale stem:[\sum_{k=0}^q \alpha_k f(\Phi(\xi_k)) J] avec la quadrature de Gauss sur les intervalles
10. affichage des résultats avec `tabulate` en utilisant le format `grid` et en arrondissant les valeurs à 9 chiffres après la virgule, une ligne correspondant à un nombre d'intervalles différent
Donc on estime que le nombre d&#8217;individus avec $1.8 \leq h \leq 1.9$ soit le 13.6 %.

On notera que la méthode de Gauss qui intègre des polynomes de degré 5 `Gauss 5` exactement nécessite seulement 8 sous intervallles pour obtenir une précision de 9 chiffres après la virgule. `Gauss 1 et 3` nécessitent 64 sous-intervalles pour obtenir une précision de 9 chiffres après la virgule. Nous avons rappelé entre parenthèses le nombre de points nécessaires par sous intervalles.

On conclut également que pour cet exemple, il est beaucoup plus efficace, précis et économique d&#8217;utiliser la méthode des points de Gauss que les autres méthodes à précision donnée.

Nous pouvons afficher une table avec les erreurs $e_i$ de convergence des méthodes d&#8217;intégration numérique pour cet exemple pour chaque nombre d&#8217;intervalles $i$


In [0]:
exact = results[-1][3]  # <1>
errors = np.abs(exact - results[:])  # <2>
for i in range(len(errors)):
    errors[i][0] = 2**i # <3>

print(tabulate(errors, headers=headers, tablefmt="grid", floatfmt=".2e"))



1. valeur exacte de l'intégrale
2. calcul des erreurs pour chaque méthode d'intégration numérique
3. nous devons corriger la première colonne pour afficher les nombres d'intervalles
Enfin nous pouvons afficher les ordre de convergence des méthodes d&#8217;intégration numérique pour cet exemple.
En effet nous pouvons calculer l&#8217;ordre de convergence de la méthode d&#8217;intégration numérique en utilisant la formule suivante:


$$
\text{ordre} = \log_2\left(\frac{e_{i-1}}{e_i}\right)
$$


In [0]:
exact = results[-1][3]  # <1>
error_prev = np.abs(exact - results[:-1])  # <2>
error = np.abs(exact - results[1:])  # <3>
orders = np.log2(error_prev / error)  # <4>
for i in range(len(orders)):
    orders[i][0] = 2**(i+1)
print(tabulate(orders, headers=headers, tablefmt="grid", floatfmt=".1f"))



1. valeur exacte de l'intégrale
2. calcul des erreurs pour chaque méthode d'intégration numérique du premier à l'avant dernier nombre d'intervalles
3. calcul des erreurs pour chaque méthode d'intégration numérique du deuxième au dernier nombre d'intervalles
4. calcul des ordres de convergence pour chaque méthode d'intégration numérique
*Important:* nous observons les ordres de convergence des méthodes d&#8217;intégration numérique exactement attendus par la théorie pour chaque méthode.
